In [ ]:
from google.colab import drive
import pandas as pd
from collections import Counter
from tqdm import tqdm
import requests
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#한국어 자연어 처리 패키지 konlpy 설치
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy
#java 환경변수 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3,145 kB]
Hit:13 http://ppa.launchpad.ne

bash: line 6: fg: no job control


CalledProcessError: ignored

In [ ]:
from konlpy.tag import Okt

In [ ]:
base = "/content/drive/MyDrive/colab/NetworkGraph"
for n in tqdm(os.listdir(base + "/data")):
  #csv 불러오기
  df = pd.read_csv(base +"/data/" + n, encoding='utf-8', sep="\t")

  #content들을 문장으로 합성
  sentence = " ".join(map(str, df["content"]))

  #형태소분석기 okt 생성
  okt=Okt()

  #명사 추출
  data = okt.nouns(sentence)

  #길이 1인 명사 제거
  nouns = Counter([i for i in data if len(i) != 1])

  #Counter.items() 수정해야함
  nouns = list(nouns.items())

  #쿼리문 생성, 결과 저장
  result = []
  for i in range(len(nouns)):
    url = "http://175.45.200.97:6161/getData?function=2&data=["
    temp = []
    for j in range(i + 1, len(nouns)):
      url += "{{\"st1\":\"{}\",\"st2\":\"{}\"}}".format(nouns[i], nouns[j])
      temp.append(nouns[j])
      if j == len(nouns) - 1 or (i == len(nouns) - 1 and j == len(nouns) - 2): continue
      url += ","
    url += "]"
    #도출 값이 0 이상인 경우에만 데이터 저장 
    result += [[nouns[i], k, l] for k, l in zip(temp, requests.get(url).json()["data"]) if l >= 0]

  #flourish 붙혀 넣기용 엑셀 파일 생성
  #1, 2, 3행 Links
  #4, 5행 Points
  for i in range(len(nouns)):
    result[i].append(nouns[i])
    result[i].append(1)
  pd.DataFrame(result).to_excel(base + "/result/" + n[:-n[::-1].find(".")-1] + ".xlsx")

  0%|          | 0/16 [00:00<?, ?it/s]


java.lang.InterruptedException: ignored